# Caffe2 Introduction

References
* https://caffe2.ai/docs/intro-tutorial.html

In [1]:
from caffe2.python import workspace, model_helper
import numpy as np

## Blobs and Workspace, Tensors

In [2]:
# Create random tensor of three dimensions
x = np.random.rand(4, 3, 2)
print(x)
print(x.shape)

workspace.FeedBlob("my_x", x)

x2 = workspace.FetchBlob("my_x")
print(x2)

[[[0.04798808 0.29514541]
  [0.47173325 0.84355219]
  [0.44440153 0.79400813]]

 [[0.04483906 0.65928398]
  [0.93235641 0.49973344]
  [0.15734319 0.01242859]]

 [[0.98126104 0.73074051]
  [0.29563129 0.26204827]
  [0.42370625 0.59264756]]

 [[0.84563836 0.56533809]
  [0.09544875 0.92463733]
  [0.3844233  0.29827066]]]
(4, 3, 2)
[[[0.04798808 0.29514541]
  [0.47173325 0.84355219]
  [0.44440153 0.79400813]]

 [[0.04483906 0.65928398]
  [0.93235641 0.49973344]
  [0.15734319 0.01242859]]

 [[0.98126104 0.73074051]
  [0.29563129 0.26204827]
  [0.42370625 0.59264756]]

 [[0.84563836 0.56533809]
  [0.09544875 0.92463733]
  [0.3844233  0.29827066]]]


## Nets and Operators

### 1. Model defintion

In [3]:
# Create the input data
data = np.random.rand(16, 100).astype(np.float32)

# Create labels for the data as integers [0, 9].
label = (np.random.rand(16) * 10).astype(np.int32)

workspace.FeedBlob("data", data)
workspace.FeedBlob("label", label)

True

In [4]:
# Create model using a model helper
m = model_helper.ModelHelper(name="my first net")

In [5]:
weight = m.param_init_net.XavierFill([], 'fc_w', shape=[10, 100])
bias = m.param_init_net.ConstantFill([], 'fc_b', shape=[10, ])

In [6]:
fc_1 = m.net.FC(["data", "fc_w", "fc_b"], "fc1")
pred = m.net.Sigmoid(fc_1, "pred")
softmax, loss = m.net.SoftmaxWithLoss([pred, "label"], ["softmax", "loss"])
print(m.net.Proto())

name: "my first net"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc1"
  name: ""
  type: "FC"
}
op {
  input: "fc1"
  output: "pred"
  name: ""
  type: "Sigmoid"
}
op {
  input: "pred"
  input: "label"
  output: "softmax"
  output: "loss"
  name: ""
  type: "SoftmaxWithLoss"
}
external_input: "data"
external_input: "fc_w"
external_input: "fc_b"
external_input: "label"



### 2. Executing

In [7]:
workspace.RunNetOnce(m.param_init_net)
workspace.CreateNet(m.net)

True

In [8]:
# Run 100 x 10 iterations
for _ in range(100):
    data = np.random.rand(16, 100).astype(np.float32)
    label = (np.random.rand(16) * 10).astype(np.int32)

    workspace.FeedBlob("data", data)
    workspace.FeedBlob("label", label)

    workspace.RunNet(m.name, 10)   # run for 10 times

In [9]:
print(workspace.FetchBlob("softmax"))
print(workspace.FetchBlob("loss"))

[[0.10693514 0.08654815 0.08640441 0.09488468 0.08006307 0.09691094
  0.11522754 0.10521664 0.11057828 0.11723107]
 [0.10524033 0.09846363 0.10429107 0.09712043 0.08648786 0.09566383
  0.09768786 0.09588178 0.11043419 0.10872909]
 [0.09116844 0.10422283 0.10762887 0.09341332 0.08236504 0.0940946
  0.11595587 0.10004821 0.10869397 0.10240878]
 [0.09754768 0.09058246 0.09380923 0.09632561 0.0779295  0.09619488
  0.11716543 0.11381692 0.10700475 0.1096236 ]
 [0.11644505 0.09216201 0.0880461  0.11386504 0.0907858  0.09495328
  0.10418115 0.0921772  0.10442338 0.10296092]
 [0.11926381 0.09341063 0.10374295 0.09161015 0.08418303 0.09291831
  0.10440864 0.09966722 0.09976219 0.11103307]
 [0.1003916  0.08444326 0.10631317 0.10027147 0.0809922  0.09353884
  0.10905166 0.10242118 0.1138364  0.10874021]
 [0.09873134 0.09193544 0.10725842 0.0965521  0.08516364 0.0902591
  0.10540079 0.1025169  0.11286637 0.10931596]
 [0.10006923 0.08418194 0.10115299 0.10232321 0.08188592 0.10007308
  0.11218643 0

### 3. Backward pass

In [10]:
m.AddGradientOperators([loss])
workspace.RunNetOnce(m.param_init_net)
workspace.CreateNet(m.net, overwrite=True)

True

In [11]:
print(m.net.Proto())

name: "my first net"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc1"
  name: ""
  type: "FC"
}
op {
  input: "fc1"
  output: "pred"
  name: ""
  type: "Sigmoid"
}
op {
  input: "pred"
  input: "label"
  output: "softmax"
  output: "loss"
  name: ""
  type: "SoftmaxWithLoss"
}
op {
  input: "loss"
  output: "loss_autogen_grad"
  name: ""
  type: "ConstantFill"
  arg {
    name: "value"
    f: 1.0
  }
}
op {
  input: "pred"
  input: "label"
  input: "softmax"
  input: "loss_autogen_grad"
  output: "pred_grad"
  name: ""
  type: "SoftmaxWithLossGradient"
  is_gradient_op: true
}
op {
  input: "pred"
  input: "pred_grad"
  output: "fc1_grad"
  name: ""
  type: "SigmoidGradient"
  is_gradient_op: true
}
op {
  input: "data"
  input: "fc_w"
  input: "fc1_grad"
  output: "fc_w_grad"
  output: "fc_b_grad"
  output: "data_grad"
  name: ""
  type: "FCGradient"
  is_gradient_op: true
}
external_input: "data"
external_input: "fc_w"
external_input: "fc_b"
external_input: "label

In [12]:
# Run 100 x 10 iterations
for _ in range(100):
    data = np.random.rand(16, 100).astype(np.float32)
    label = (np.random.rand(16) * 10).astype(np.int32)

    workspace.FeedBlob("data", data)
    workspace.FeedBlob("label", label)

    workspace.RunNet(m.name, 10)   # run for 10 times

In [13]:
print(workspace.FetchBlob("softmax"))
print(workspace.FetchBlob("loss"))

[[0.09061375 0.08890209 0.07852141 0.09371094 0.07721997 0.12769563
  0.11973359 0.11358465 0.09014037 0.11987755]
 [0.10043311 0.10695577 0.08506128 0.08558637 0.08794773 0.10406561
  0.11723498 0.10433999 0.08258724 0.12578791]
 [0.09774163 0.09779619 0.08986595 0.08786304 0.09102538 0.10312196
  0.11283018 0.09987509 0.08569832 0.13418224]
 [0.09003852 0.1105454  0.09401692 0.08878256 0.08762946 0.1017843
  0.1188451  0.09826504 0.08533151 0.12476113]
 [0.08791647 0.09521629 0.09561692 0.10444915 0.08983332 0.10842929
  0.11971044 0.09491967 0.08442746 0.11948101]
 [0.10020016 0.10253922 0.08672389 0.0940804  0.08732906 0.11940424
  0.11585031 0.09984045 0.08283658 0.11119567]
 [0.09474728 0.09664725 0.09139288 0.0931303  0.09788895 0.10375027
  0.12170776 0.09591308 0.08846743 0.11635488]
 [0.09476154 0.09302573 0.09776563 0.09404112 0.08540931 0.10567707
  0.11527937 0.10360312 0.08961114 0.12082597]
 [0.0887873  0.09437061 0.09314491 0.09422383 0.08335888 0.11679742
  0.12219154 